# Inhaltsübersicht EDA

### 0. Datensatz laden und bereinigen mit data_preparation.py (neu machen, wenn Funktion abgeschlossen)
- kurze Beschreibung der Daten (Ursprung, Umfang)
- Funktion data_import()
- Funktion data_cleaning()

### 1. Überblick über den Datensatz
- Spaltenbeschreibung/Datentypen: Funktion overview()
- Fehlende Werte (NaN-Anteile)
- Entfernen von Variablen mit mehr als 50% fehlenden Werten

### 2. Deskriptive Statistik & Ausreißer-Handling
#### 2.1 Schadstoffe (einzeln)
- describe()-Tabelle für Schadstoffe
- Stripplots zur Ausreißererkennung für alle Schadstoffe
- Vergleich von Mean und Median und Schiefe (skew)
- Zellenweise Ersetzung durch NaN (statt Zeilenlöschung)
- Dokumentaton der Schwellenwerte (Cutoff) und Filterlogik
- Tabelle mit Anzahl der entfernten Werte

#### 2.2 Wetter (Schleife)
- describe()-Tabelle für Wetter
- Stripplots und Kennzahlen zur Ausreißererkennung für alle Wettervariablen
- Festlegen der Schwellenwerte für Ausreißer, Ersetzen durch NaN
- Dokumentaton der Schwellenwerte (Cutoff) und Filterlogik
- Tabelle mit Anzahl der entfernten Werte

### 3. Korrelationsanalysen
#### 3.1 Korelationsmatrix
- Korrelationsmatrix (Pearson)
- Identifikation und Dokumentation der stärksten Korrelationen
- Empfehlung zur Variablenselektion (evtl. PCA)

#### 3.2 Pairplots der wichtigsten Feature-Paare
- Berechnung und Interpretation von Pairplots
- Überprüfen der Art des Zusammenhangs (linear oder nicht) mit linearer Regression und LOWESS (Local Weighted Scatterplot Smoothing)
- Individuell getestete Feature-Paare: PM10-PM25, NO₂-PM10, NO₂-PM25, Humidity-PM25, O3-Tavg, Tmin-Dew, Pres-Dew, Pres-Tmin 
- Abschließende Bewertung der Korrelationen und der Datenstruktur

# 0. Datensatz laden

Der Datensatz enthält Messungen zur Luftqualität sowie Wetter- und Bevölkerungdaten aus den Jahren 2014-2024, aggregiert aus verschiedenen Quellen mit einem anfänglichen Gesamtumfang von ca. 14 Mio Datenpunkten (s. README.md --> Datenquellen)
Um alle Daten in einem Pandas-Dataframe bearbeiten zu können, werden zunächst die Datensätze mit der Funktion data_import() aggregiert und anschlie0end mit der Funktion data_cleaning() so bereinigt, dass z.B. redundante Spalten entfernt werden. Die Funktionen und ihre Unterfunktionen finden befinden sich im Skript data_preparation.py.

In [1]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import skew
%matplotlib inline

In [2]:
# Settings for displaying floats
pd.set_option('display.float_format', '{:,.2f}'.format)

In [3]:
df = pd.read_csv("./data/cleaned_air_quality_data_2025_03_25.csv")
df.head()

,Year,Month,Day,Country,City,Latitude,Longitude,Co,No2,O3,...,So2,Dew,Humidity,Tavg,Tmin,Tmax,Prcp,Wdir,Wspd,Pres
0,2014,12,29,AT,Graz,47.07,15.45,0.10,9.00,NaN,...,1.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2014,12,29,AT,Innsbruck,47.26,11.39,0.10,25.60,NaN,...,1.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2014,12,29,AT,Linz,48.31,14.29,0.10,14.20,NaN,...,2.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2014,12,29,AT,Salzburg,47.80,13.04,0.10,21.10,NaN,...,2.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2014,12,29,AT,Vienna,48.21,16.37,0.10,9.00,NaN,...,2.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Zelle löschen, wenn cleaning-Funktion das übernimmt!

df.columns

# Some redundant columns still need to be removed from df manually: Temperature, Pressure, Wind-speed

Index(['Year', 'Month', 'Day', 'Country', 'City', 'Latitude', 'Longitude',
       'Co', 'No2', 'O3', 'Pm10', 'Pm25', 'So2', 'Dew', 'Humidity', 'Tavg',
       'Tmin', 'Tmax', 'Prcp', 'Wdir', 'Wspd', 'Pres'],
      dtype='object')

In [6]:
df.shape

(1695041, 22)

# 1. Überblick über den Datensatz

- Überblicksfunktion .overview()
- Fehlende Werte ausloten und Spalten mit < 50% Daten entfernen.
- Nach dem kompletten Entfernen aller Spalten mit < 50% Daten umfasst der Datensatz **1.695.041 Zeilen und 22 Spalten**.

In [7]:
# Function to create overview for dtypes, missing values, unique values

def overview(df):
    '''
    Erstelle einen Überblick über einige Eigenschaften der Spalten eines DataFrames.
    VARs
        df: Der zu betrachtende DataFrame
    RETURNS:
        None
    '''
    display(pd.DataFrame({'dtype': df.dtypes,
                          'total': df.count(),
                          'missing': df.isna().sum(),
                          'missing%': df.isna().mean()*100,
                          'n_uniques': df.nunique(),
                          'uniques%': df.nunique()/df.shape[0]*100,
                          'uniques': [df[col].unique() for col in df.columns]
                         }))


In [8]:
overview(df)

,dtype,total,missing,missing%,n_uniques,uniques%,uniques
Year,int64,1695041,0,0.00,12,0.00,"[2014, 2015, 2016, 2017, 2018, 2019, 2020, 202..."
Month,int64,1695041,0,0.00,12,0.00,"[12, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]"
Day,int64,1695041,0,0.00,31,0.00,"[29, 30, 31, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11..."
Country,object,1695041,0,0.00,95,0.01,"[AT, AU, BE, BO, BR, CA, CH, CL, CN, CO, CY, C..."
City,object,1695041,0,0.00,616,0.04,"[Graz, Innsbruck, Linz, Salzburg, Vienna, Bris..."
Latitude,float64,1692212,2829,0.17,617,0.04,"[47.06667, 47.26266, 48.30639, 47.79941, 48.20..."
Longitude,float64,1692212,2829,0.17,616,0.04,"[15.45, 11.39454, 14.28611, 13.04399, 16.37208..."
Co,float64,1063073,631968,37.28,988,0.06,"[0.1, 1.9, 2.1, nan, 2.3, 2.0, 3.4, 5.5, 3.8, ..."
No2,float64,1408527,286514,16.90,1271,0.07,"[9.0, 25.6, 14.2, 21.1, 4.6, 0.7, nan, 6.5, 4...."
O3,float64,1320290,374751,22.11,1496,0.09,"[nan, 3.7, 4.3, 12.8, 15.2, 21.0, 17.6, 6.1, 2..."


In [ ]:
# Sort missing values (descending)

missing_percent = df.isna().mean() * 100
missing_percent_sorted = missing_percent.sort_values(ascending=False)

print(missing_percent_sorted)


In [ ]:
# Ursprünglich wurden hier Spalten mit >50% fehlenden Werten entfernt.
# Der aktuelle Datensatz ist vollständig genug – daher ist dieser Schritt derzeit nicht notwendig.
# Falls sich das ändert, kann der folgende Codeblock wieder aktiviert werden:

# # Remove all columns with more than 50% missing values
# # Note: This removes population data entirely!

# # Anzahl der Spalten vor dem Entfernen merken
# original_columns = df.shape[1]

# df = df.loc[:, missing_percent <= 50]

# # Print what has changed
# print(f"Anzahl der entfernten Spalten: {original_columns - df.shape[1]}")
# removed = missing_percent[missing_percent > 50].index
# print("Entfernte Spalten:", list(removed))
# print("Übrige Spalten:", df.columns)

# df.shape()

# 2. Deskriptive Statistik und Ausreißerentfernung

Vor der systematischen Ausreißeranalyse ist es sinnvoll, kurz die Art der Daten zu reflektieren:
Wetterdaten (wie Temperatur, Luftdruck oder Windgeschwindigkeit) werden seit Jahrzehnten weltweit standardisiert erfasst. Die verwendeten Messinstrumente sind technisch ausgereift, gut kalibriert und liefern in der Regel konsistente Ergebnisse. Aus diesem Grund ist hier nur mit wenigen Ausreißern oder Platzhalterwerten zu rechnen.

Bei Schadstoffmessungen (wie PM₂.₅, NO₂ oder O₃) sieht die Situation anders aus. Diese Daten werden häufig mit komplexeren Sensoren erhoben, die empfindlicher auf Umwelteinflüsse oder Wartungsprobleme reagieren. Außerdem sind Schadstoffmessstationen nicht flächendeckend verfügbar, was zu uneinheitlicher Datenqualität führen kann. In der Praxis zeigt sich das häufig durch auffällige Ausreißerwerte oder symbolische Platzhalter wie 500 oder 999, die als Maximalwert oder Fehlwert dienen.

Auf Basis dieser Überlegungen erwarten wir bei der folgenden Analyse:

- Mehr auffällige Werte in den Schadstoffvariablen
- Weniger (aber nicht zwingend keine) Ausreißer bei den Wetterdaten

Diese Hypothese bestätigt sich auch in der Auswertung.

Entsprechend dieser Erwartung wurde das methodische Vorgehen angepasst:

- Die Schadstoffvariablen wurden einzeln analysiert – mit Stripplots, Schiefe-Werten und einer individuellen Beurteilung der Ausreißerstruktur.

- Bei den Wettervariablen wurde – unter Berücksichtigung ihrer geringeren Anfälligkeit für Messfehler – eine automatisierte Analyse mittels Schleife durchgeführt, die Kennzahlen berechnet und visualisiert, um dennoch potenzielle Ausreißer strukturiert erkennen und ggf. entfernen zu können.

Die Entscheidungen, wo jeweils der Schwellenwerte  für die Ausreißerentfernung gesetzt wird, sind nicht rein statistisch begründet (z.B. 99,99-Quantil als Cutoff), sondern werden für jedes Feature individuell getroffen. Ziel der Ausreißerentfernung ist es, sachlich unmögliche Werte zu entfernen, die natürliche Varianz zu erhalten und einen repräsentativen Datensatz für folgende Modellierungen zu schaffen.

Methode zur Ausreißerentfernung: Zur Datenbereinigung wurden extreme Ausreißer in den Schadstoffwerten nicht durch das Entfernen ganzer Zeilen, sondern durch das gezielte Ersetzen der jeweiligen Zellen mit NaN behandelt. So bleiben andere valide Messwerte in der gleichen Zeile erhalten.

# Analyse der Schadstoffdaten

In [ ]:
# Pollutants

pollutant_cols = ["Co", "Pm25", "Pm10", "So2", "No2", "O3"]
df[pollutant_cols].describe().T


Die Messwerte, die als Maxima angegeben sind (Co = 500 oder Pm25 = 999) lassen Platzhalter für fehlende oder ungültige Daten vermuten. Solche Werte können die Verteilung erheblich verzerren und sollten im Folgenden genau in Augenschein genommen werden.

Damit die Anzahl der entfernten Ausreißer ermittelt werden kann, muss vor der Aureißerentfernung deren Anzahl ermittelt und gespeichert werden:

In [ ]:
# Vor der Bereinigung: fehlende Werte zählen
missing_before = df.isna().sum()

# Gesamtgröße des df speichern, um später den Prozentsatz an durch Ausreißerentfernung verlorenen Werten zu berechnen
total_values_before = df.shape[0] * df.shape[1]

## Mittelgroße Feinstaubpartikel (Pm25)

"Die als Feinstaub (PM2,5) bezeichnete Staubfraktion enthält 50% der Teilchen mit einem Durchmesser von 2,5 µm, einen höheren Anteil kleinerer Teilchen und einen niedrigeren Anteil größerer Teilchen." (https://www.umweltbundesamt.at/umweltthemen/luft/luftschadstoffe/staub/pm25)


In [ ]:
sns.stripplot(x=df["Pm25"], size=2, jitter=0.3)
plt.title("PM2.5 Einzelwerte")
plt.xlabel("PM2.5-Konzentration (µg/m³)")
plt.grid(True);

Bei der visuellen Analyse der PM2.5-Messwerte fällt eine ungewöhnliche Häufung im Bereich zwischen 816 und 834 µg/m³ auf, begleitet von vereinzelten extrem hohen Werten bei 981 und 999 µg/m³. Diese Werte liegen deutlich außerhalb des realistischen Messbereichs und deuten auf Messfehler, Platzhalterwerte oder systembedingte Übertragungsfehler hin. Um die Analyse nicht zu verzerren, werden daher alle PM2.5-Werte ab 816 µg/m³ ausgeschlossen.

In [ ]:
# PM2.5 >= 816
df.loc[df["Pm25"] >= 816, "Pm25"] = np.nan

In [ ]:
sns.histplot(df["Pm25"], bins=50)

In [ ]:
mean_pm25 = df["Pm25"].mean()
median_pm25 = df["Pm25"].median()
skew_pm25 = skew(df["Pm25"].dropna())

print(f"Mittelwert (mean): {mean_pm25:.2f}")
print(f"Median:            {median_pm25:.2f}")
print(f"Schiefe (Skewness): {skew_pm25:.2f}")

### Verteilung der bereinigten PM2.5-Werte

Die PM2.5-Werte zeigen eine deutlich rechtsschiefe Verteilung. Der Mittelwert liegt bei 53,19 µg/m³, während der Median nur 42,00 µg/m³ beträgt. Zusätzlich ergibt die Berechnung der Schiefe (Skewness) einen Wert von 2,26, was auf eine starke Asymmetrie hindeutet.
Diese rechtsschiefe Verteilung bedeutet, dass der Großteil der Messwerte im unteren Bereich liegt, während einige wenige sehr hohe Werte den Mittelwert nach oben ziehen.

Um die Analyse nicht durch technisch bedingte Ausreißer zu verzerren, wurden alle PM2.5-Werte ab 816 µg/m³ ausgeschlossen, da sie deutlich außerhalb des natürlichen Messbereichs liegen und teilweise systematisch auftraten (Platzhalterwerte).

## Kohlenmonoxid (CO)

"Kohlenmonoxid (CO) ist ein farb-, geruch- und geschmackloses Gas, das bei der unvollständigen Verbrennung von Brenn- und Treibstoffen entsteht. Es bildet sich, wenn bei Verbrennungsprozessen zu wenig Sauerstoff zur Verfügung steht. In höheren Konzentrationen wirkt CO als starkes Atemgift." (https://www.umweltbundesamt.de/themen/luft/luftschadstoffe-im-ueberblick/kohlenmonoxid)

In [ ]:
sns.stripplot(x=df["Co"], size=2, jitter=0.3)
plt.title("CO Einzelwerte")
plt.xlabel("CO-Konzentration (µg/m³)")
plt.grid(True);

In [ ]:
mean_co = df["Co"].mean()
median_co = df["Co"].median()
skew_co = skew(df["Co"].dropna())

print(f"Mittelwert (mean): {mean_co:.2f}")
print(f"Median:            {median_co:.2f}")
print(f"Schiefe:           {skew_co:.2f}")

In [ ]:
# CO > 300 oder CO == 500 → als NaN
df.loc[df["Co"] >= 300, "Co"] = np.nan

### Verteilung der bereinigten CO-Werte

Die Verteilung der CO-Messwerte ist stark rechtsschief, mit einem Mittelwert von 4.74 µg/m³, einem Median von 3.40 µg/m³ und einer Schiefe von 29.64. Dies deutet auf extreme Ausreißer hin, die den Mittelwert erheblich verzerren. Zusätzlich tritt eine auffällige Häufung exakt bei 500 µg/m³ auf, was sehr wahrscheinlich ein technischer Platzhalterwert ist.

Um die Aussagekraft der Analyse zu erhöhen, wurden daher alle Werte ab 300 µg/m³ ausgeschlossen. Dieser Grenzwert liegt deutlich oberhalb der Hauptverteilung und entfernt technische Ausreißer, aber auch extreme realistische Einzelwerte.

## Stickstoffdioxid (No2)

"Stickstoffdioxid (NO2) ist ein ätzendes Reizgas, es schädigt unmittelbar das Schleimhautgewebe im gesamten Atemtrakt und kann auch die Augen reizen. Stickstoffdioxid zeigt eine stärkere schädliche Wirkung als Stickstoffmonoxid (NO), weshalb Stickstoffdioxid im Zentrum der Bemühungen um saubere Luft steht. [...] Die mittelbare Wirkung des Stickstoffdioxids auf die menschliche Gesundheit besteht in seiner Eigenschaft als Vorläufersubstanz für die Bildung von Feinstaub." (https://www.umweltbundesamt.de/service/uba-fragen/wie-wirken-sich-stickstoffoxide-auf-die-menschliche)

In [ ]:
sns.stripplot(x=df["No2"], size=2, jitter=0.3)
plt.title("NO₂ Einzelwerte")
plt.xlabel("NO₂-Konzentration (µg/m³)")
plt.grid(True);

In [ ]:
mean_no2 = df["No2"].mean()
median_no2 = df["No2"].median()
skew_no2 = skew(df["No2"].dropna())

print(f"Mittelwert (mean): {mean_no2:.2f}")
print(f"Median:            {median_no2:.2f}")
print(f"Schiefe:           {skew_no2:.2f}")

In [ ]:
# NO2 >= 300 oder == 500
df.loc[df["No2"] >= 300, "No2"] = np.nan

### Bereinigung der NO₂-Werte
Die NO₂-Werte zeigen eine stark rechtsschiefe Verteilung, mit einem Mittelwert von 10.55 µg/m³, einem Median von 8.40 µg/m³ und einer Schiefe von 10.08. Die große Mehrheit der Messwerte liegt unterhalb von 120 µg/m³, während einzelne Ausreißer bis 425 µg/m³ reichen. 

Zusätzlich tritt eine auffällige Häufung bei 500 µg/m³ auf, was sehr wahrscheinlich auf technische Platzhalterwerte zurückzuführen ist.
Um die Analyse nicht durch extreme Einzelwerte zu verzerren, wurden alle NO₂-Werte ab 300 µg/m³ ausgeschlossen.

## Schwefeldioxid (So2)

"Schwefeldioxid (SO2) ist ein farbloses, stechend riechendes, wasserlösliches Gas, das Mensch und Umwelt beeinträchtigt. In der Atmosphäre aus Schwefeldioxid entstehende Sulfatpartikel tragen zudem zur Belastung mit Feinstaub (PM10) bei." (https://www.umweltbundesamt.de/themen/luft/luftschadstoffe-im-ueberblick/schwefeldioxid)

In [ ]:
sns.stripplot(x=df["So2"], size=2, jitter=0.3)
plt.title("SO₂ Einzelwerte")
plt.xlabel("SO₂-Konzentration (µg/m³)")
plt.grid(True);

In [ ]:
mean_so2 = df["So2"].mean()
median_so2 = df["So2"].median()
skew_so2 = skew(df["So2"].dropna())

print(f"Mittelwert (mean): {mean_so2:.2f}")
print(f"Median:            {median_so2:.2f}")
print(f"Schiefe:           {skew_so2:.2f}")

Es zeigt sich eine auffällige Punktelinie zwischen 300 und 400, die einen technischen Maximalwert bestimmter Messgeräte vermuten lässt. Wo genau liegt diese Linie?

In [ ]:
df["So2"].value_counts().sort_index(ascending=False).head(10)

In [ ]:
# SO2 >= 352 oder == 500
df.loc[df["So2"] >= 352, "So2"] = np.nan

### Bereinigung der SO₂-Werte
Die Variable So2 misst die tägliche Schwefeldioxidkonzentration in µg/m³. Die Verteilung ist stark rechtsschief (Skewness = 25.75) mit einem Median von 2.5 µg/m³ und einem Mittelwert von 3.98 µg/m³.

Der Großteil der Werte liegt im Bereich von 0 bis 70 µg/m³, wobei sich oberhalb dieses Bereichs eine kontinuierlich abnehmende Streuung zeigt.

Ab etwa 220 µg/m³ tritt eine leichte Häufung auf, bei 352 µg/m³ eine weitere, und bei 500 µg/m³ zeigt sich eine deutliche Punktelinie – vermutlich ein technischer Maximalwert oder Platzhalter.

Zur Verbesserung der Datenqualität und Modellierbarkeit wurde ein Cutoff bei 352 µg/m³ gesetzt.
Alle Werte oberhalb dieses Grenzwerts wurden durch NaN ersetzt.

## Analyse und Bereinigung der O3-Werte

"Das farblose und giftige Gas Ozon ist eines der wichtigsten Spurengase in der Atmosphäre. [...] In Bodennähe auftretendes Ozon wird nicht direkt freigesetzt, sondern bei intensiver Sonneneinstrahlung durch komplexe photochemische Prozesse aus Vorläuferschadstoffen − überwiegend Stickstoffoxiden und flüchtigen organischen Verbindungen − gebildet. Ozon wird deshalb als sekundärer Schadstoff bezeichnet." (https://www.umweltbundesamt.de/themen/luft/luftschadstoffe-im-ueberblick/ozon)

In [ ]:
sns.stripplot(x=df["O3"], size=2, jitter=0.3)
plt.title("O3 Einzelwerte")
plt.xlabel("O3-Konzentration (µg/m³)")
plt.grid(True);

In [ ]:
mean_o3 = df["O3"].mean()
median_o3 = df["O3"].median()
skew_o3 = skew(df["O3"].dropna())

print(f"Mittelwert (mean): {mean_o3:.2f}")
print(f"Median:            {median_o3:.2f}")
print(f"Schiefe:           {skew_o3:.2f}")

In [ ]:
df["O3"].value_counts().sort_index(ascending=False).head(20)

In [ ]:
# O3 >= 323.5 oder == 500
df.loc[df["O3"] >= 323.5, "O3"] = np.nan

### Bereinigung der Ozon-Werte (O₃)
Die Ozonwerte (O3) zeigen eine stark rechtsschiefe Verteilung mit einer Skewness von 13.71. Der Median liegt bei 19.90 µg/m³, der Mittelwert bei 20.46 µg/m³ – was nahe beieinander liegt, aber durch einige hohe Ausreißer leicht verzerrt wird.

Im oberen Bereich der Verteilung ist eine ausgeprägte Punktelinie bei 395 µg/m³ erkennbar, mit 36 identischen Messwerten. Zudem zeigt sich eine große Häufung bei 500 µg/m³ mit 536 Einträgen – ein starkes Indiz für technische Grenzwerte oder Platzhalterwerte.

Zwischen 323.5 µg/m³ und 381 µg/m³ besteht eine auffällige Lücke, was darauf hinweist, dass oberhalb von 323.5 keine kontinuierliche natürliche Streuung mehr vorhanden ist.

→ Es wurde ein Cutoff bei 323.5 µg/m³ gesetzt.
Alle Werte oberhalb dieses Grenzwerts wurden durch NaN ersetzt.

## Analyse und Bereinigung der Pm10-Werte

"Als Feinstaub (⁠PM10⁠) bezeichnet man Partikel mit einem aerodynamischen Durchmesser von weniger als 10 Mikrometer (µm). Der größte Teil der anthropogenen Feinstaubemissionen stammt aus Verbrennungsvorgängen (Kfz-Verkehr, Gebäudeheizung) und Produktionsprozessen inkl. Schüttgutumschlag. [...] Feinstaub wird nicht nur direkt emittiert (primäre Partikel) sondern bildet sich auch aus Vorläuferstoffen (unter anderem aus Schwefeldioxid, Stickstoffoxid und Ammoniak) in der Atmosphäre (sekundäre Partikel)." (https://www.umweltbundesamt.de/daten/luft/luftschadstoff-emissionen-in-deutschland/emission-von-feinstaub-der-partikelgroesse-pm10#was-ist-feinstaub)

In [ ]:
sns.stripplot(x=df["Pm10"], size=2, jitter=0.3)
plt.title("Pm10 Einzelwerte")
plt.xlabel("Pm10-Konzentration (µg/m³)")
plt.grid(True);

In [ ]:
df["Pm10"].describe()

In [ ]:
mean_pm10 = df["Pm10"].mean()
median_pm10 = df["Pm10"].median()
skew_pm10 = skew(df["Pm10"].dropna())

print(f"Mittelwert (mean): {mean_pm10:.2f}")
print(f"Median:            {median_pm10:.2f}")
print(f"Schiefe:           {skew_pm10:.2f}")

In [ ]:
# Wo beginnt die Häufung bei der breiten Linie im Ausreißerbereich (ca. 820-840)?

df["Pm10"].value_counts().loc[lambda x: x.index > 850].sort_index()


In [ ]:
# "Datenmauer" plotten (nicht wirklich nötig, aber "Nice to have" für Leute, die gerne Visualisierungen mögen...)

high_values = df["Pm10"][df["Pm10"] > 800]
sns.histplot(high_values, binwidth=1)
plt.title("Häufigkeit der PM10-Werte > 800")
plt.xlabel("PM10-Wert (µg/m³)")
plt.ylabel("Anzahl")
plt.grid(True)
plt.show()


In [ ]:
# PM10 >= 867
df.loc[df["Pm10"] >= 867, "Pm10"] = np.nan


### Bereinigung der PM10-Werte
Die PM10-Werte zeigen eine deutlich rechtsschiefe Verteilung mit einem Mittelwert von 32.43 µg/m³, einem Median von 24.00 µg/m³ und einer Schiefe von 7.17. Der größte Teil der Messwerte liegt unterhalb von 200 µg/m³, mit einer kontinuierlich abnehmenden Dichte bis in den Bereich von ca. 850 µg/m³.

Eine detaillierte Analyse der Einzelwertverteilung ergab jedoch eine auffällige Häufung von Messwerten im Bereich 867–882 µg/m³, insbesondere 40 Messungen mit dem identischen Wert 867. Diese systematische Häufung spricht stark für einen technisch bedingten Fehler oder künstlich begrenzte Wertebereiche. Zusätzlich tritt der Platzhalterwert 999 33 Mal auf.

Um die Analyse nicht durch diese systematischen Verzerrungen zu beeinflussen, wurden alle PM10-Werte ab 867 µg/m³ aus dem Datensatz ausgeschlossen.

### Vergleich Anzahl fehlender Werte vor und nach der Ausreißerentfernung (Schadstoffe)

In [ ]:
missing_after = df.isna().sum()
outliers_removed = missing_after - missing_before

outlier_table = pd.DataFrame({
    "Missing Before": missing_before,
    "Missing After": missing_after,
    "Outliers Replaced": missing_after - missing_before
})

# Nur Zeilen anzeigen, in denen tatsächlich Ausreißer ersetzt wurden - alos die Schadstoff-Spalten
outlier_table_filtered = outlier_table[outlier_table["Outliers Replaced"] != 0]

outlier_table_filtered

In [ ]:
# Wieviele Daten (%) wurden durch Ausreißerentfernung verloren?
missing_before_total = missing_before.sum()
missing_after_total = missing_after.sum()
outliers_total = missing_after_total - missing_before_total
outlier_percent = outliers_total / total_values_before * 100

print(f"Anzahl ersetzter Ausreißer: {outliers_total:,}")
print(f"Prozentualer Anteil der ersetzten Ausreißer: {outlier_percent:.4f}%")

Durch das Entfernen klar identifizierter Ausreißer **in den Schadstoffvariablen** wurden lediglich 0.0041 % der Datenpunkte ersetzt.
Der Datensatz wurde dadurch kaum reduziert, ist aber nun robuster gegenüber Verzerrungen, was insbesondere für Machine-Learning-Modelle, Visualisierungen und Clusteranalysen von Vorteil ist.

# Analyse und Bereinigung der Wettervariablen
Analog zum Vorgehen bei den Schadstoffen werden im Folgenden alle Wettervariablen untersucht und ggf. Platzhalterwerte und starke Ausreißer durch NaN ersetzt.

Anmerkung: Man KÖNNTE auch nur die Platzhalterwerte durch NaN ersetzten und die Ausreißer gesondert kennzeichen. Aber darauf wird jetzt erst einmal verzichtet.

Die Wettervariablen werden in einer Schleife am Stück abgehandelt. Da das Setzen der Grenzwerte und das Entfernen von Ausreißern für jede Variable individuell entscheiden werden sollte, um möglichst alle realistischen Werte zu erhalten, wir dieser Schritt nachgelagert.


In [ ]:
# Save percentage of missing values before manipulatiing weather data
missing_before_weather = df.isna().sum()
total_values_before_weather = df.shape[0] * df.shape[1]


In [ ]:
df[["Tavg", "Tmin", "Tmax", "Humidity", "Wspd", "Wdir", "Pres", "Prcp", "Dew"]].describe()


## Auswertung von describe()

| Spalte | Bedingung für NaN                     | Begründung |
|:-------|:--------------------------------------|:-----------|
|Tavg    |                                       |                              |
|Tmin    |                                       |                          |
|Tmax	 |Tmax > 60	                             |gemessenes Maximum weit über weltweit beobachtete Temperaturen|
|Humidity|Humidity < 0 oder > 100                |physikalisch unmöglich / Platzhalter |
|Wspd	 |erst visuell prüfen, dann ggf. > 150   |Hurrikan-Grenze, höhere Werte für einzelen Böen möglich aber selten |
|Wdir    |<0 oder >360                           |Windrichtung wird in Winkelgrad gemessen |
|Pres	 |Pres > 1100	                         |Maximum ungewöhnlich hoch → Messfehler möglich |
|Prcp	 |                                       |plausible Naturverteilung, visuell prüfen
|Dew	 |ggf. ab –70 prüfen                     |tendenziell realistisch, vorsichtiger Umgang


# Schleife für Wettervariablen

In [ ]:
weather_columns = ["Tavg", "Tmin", "Tmax", "Humidity", "Wspd", "Wdir", "Pres", "Prcp", "Dew"]

for col in weather_columns:
    print(f"\n📊 Analyse für: {col}")
    
    # Stripplot
    plt.figure(figsize=(10, 2))
    sns.stripplot(x=df[col], size=2, jitter=0.3)
    plt.title(f"{col} – Einzelwerte")
    plt.grid(True)
    plt.xlabel(f"{col}-Wert")
    plt.show()
    
    # Statistische Kennzahlen
    mean_val = df[col].mean()
    median_val = df[col].median()
    skew_val = df[col].skew()
    min_val = df[col].min()
    max_val = df[col].max()
    
    print(f"Mean:   {mean_val:.2f}")
    print(f"Median: {median_val:.2f}")
    print(f"Skew:   {skew_val:.2f}")
    print(f"Min:    {min_val:.2f}")
    print(f"Max:    {max_val:.2f}")
    
    # Cutoff-Vorschlag (nur zur Orientierung)
    cutoff = df[col].quantile(0.999)
    print(f"99.9%-Quantil (statistical suggestion only!): {cutoff:.2f}")


### Cutoffbestimmung und Ausreißerentfernung für Wettervariablen

Für Variablen, bei denen der Stripplot keine eindeutige Auskunft gibt, wird der Cutoff-Wert durch einen Blick auf die betreffenden Messwerte bestimmt:

In [ ]:
# Cutoff für Tmax genauer bestimmen; oberste 10 Messwerte anschauen:
df["Tmax"].value_counts().sort_index(ascending=False).head(10)

# --> Cutoff bei 60 Grad

In [ ]:
# Wie entsteht der "gestrichelte" Eindruck im Stripplot für den Taupunkt? Gibt es mehrhheitlich ganzzahlige Messwerte?
df["Dew"].dropna().apply(lambda x: x.is_integer()).value_counts()

# --> Vermutlich geben die meisten Messgeräte nur ganzzahlige Werte aus

In [ ]:
# Tmax: realistische Obergrenze bei 60°C
df.loc[df["Tmax"] > 60, "Tmax"] = np.nan

# Humidity: alles < 0 oder > 100 ist physikalisch nicht (bzw. über 100% nur SEHR kurzzeitig) möglich
df.loc[(df["Humidity"] < 0) | (df["Humidity"] > 100), "Humidity"] = np.nan

# Wspd: Werte > 150 km/h (Orkangrenze) sind extrem selten
df.loc[df["Wspd"] > 150, "Wspd"] = np.nan

# Pres: zwei Ausreißer über 1100 hPa, vermutlich technisches Problem
df.loc[df["Pres"] > 1100, "Pres"] = np.nan

# Prcp: 2 extreme Ausreißer, im Prinzip möglich, aber für Modelle ungünstig und für den Gesamtdatensatz nicht repräsentativ
df.loc[df["Prcp"] > 400, "Prcp"] = np.nan

# Dew: Werte von -40 bis 32 sind zu erwarten; mit diesen Schwellenwerten werden nur wenige Extremwerte entfernt
df.loc[(df["Dew"] < -40) | (df["Dew"] > 32), "Dew"] = np.nan

## Ausführliche Beschreibung der Wettervariablen

🌡️ Tavg – Durchschnittstemperatur

Die Variable Tavg beschreibt die durchschnittliche tägliche Temperatur. Die Verteilung ist leicht linksschief mit einem Skewness-Wert von –0.38. Der Mittelwert beträgt 15.38 °C, der Median liegt bei 15.90 °C, was auf eine weitgehend symmetrische Verteilung mit leichtem Übergewicht kälterer Temperaturen hinweist.

Ein Blick auf den Stripplot zeigt, dass sich alle Werte im realistisch meteorologischen Bereich befinden:

- Das Minimum liegt bei ca. –42 °C, was in sehr kalten Regionen (z. B. in Sibirien oder Teilen Kanadas) durchaus auftreten kann.
- Das Maximum liegt bei ca. 50 °C, was mit bekannten Hitzerekorden weltweit vereinbar ist.

Das 99.9%-Quantil liegt bei 38.30 °C. Dennoch wurde bewusst kein Grenzwert (Cutoff) gesetzt, da keine eindeutig unrealistischen Ausreißer erkennbar sind.

→ Es wurden keine Werte entfernt oder als Ausreißer markiert.

---

🌡️ Tmin – Tagestiefsttemperatur

Die Variable Tmin beschreibt die tägliche Tiefsttemperatur. Die Verteilung zeigt mit einer Schiefe von –0.32 eine leichte Linksschiefe, was auf ein geringfügiges Übergewicht sehr kalter Tage hinweist.
Der Mittelwert beträgt 10.82 °C, der Median 11.00 °C, was auf eine insgesamt symmetrische Verteilung ohne nennenswerte Verzerrungen schließen lässt.

Der Stripplot zeigt eine natürliche, kontinuierliche Verteilung der Messwerte.

- Das Minimum liegt bei –46 °C, was in kalten Klimazonen realistisch ist.
- Das Maximum bei 39 °C ist ungewöhnlich hoch für eine Tiefsttemperatur, aber unter besonderen Bedingungen (z. B. Hitzewellen) möglich.
- Das 99.9%-Quantil liegt bei 32.4 °C, wodurch nur sehr wenige extrem hohe Werte überhaupt betroffen wären.

Insgesamt zeigen sich keine auffälligen Ausreißer oder technische Artefakte.

→ Es wurden keine Werte entfernt oder ersetzt.

---

🌡️ Tmax – Tageshöchsttemperatur

Die Variable Tmax beschreibt die tägliche Höchsttemperatur.
Die Verteilung ist mit einem Skewness-Wert von –0.37 leicht linksschief.
Der Mittelwert beträgt 20.38 °C, der Median liegt bei 21.10 °C – die Verteilung ist insgesamt gut balanciert.

Der Stripplot zeigt einen realistischen Verlauf im Bereich zwischen ca. –37 °C und 50 °C, wobei sich oberhalb von 50 °C vereinzelte Werte häufen. Eine genauere Prüfung der 20 höchsten Werte ergab jedoch drei offensichtliche Ausreißer:
→ 60 °C, 76 °C und 87 °C – allesamt deutlich über bekannten meteorologischen Rekorden.

→ Als Grenzwert wurde daher ein Cutoff bei 60 °C gesetzt. Drei Werte oberhalb dieses Grenzwerts wurden durch NaN ersetzt.

---

💧 Humidity – Relative Luftfeuchtigkeit

Die Variable Humidity beschreibt die relative Luftfeuchtigkeit in Prozent.
Die Verteilung ist extrem linksschief mit einer Skewness von –27.84, was auf starke Ausreißer im negativen und sehr hohen Bereich hindeutet.

Die realistischen Messwerte liegen zwischen 0 % und 100 %.
Der Stripplot zeigt jedoch vereinzelte, stark auffällige Werte:

- Das Minimum beträgt –2671.1 %, was physikalisch unmöglich ist
- Das Maximum liegt bei 999.9 %, höchstwahrscheinlich ein Platzhalterwert

→ Es wurde daher ein Grenzbereich von 0 % bis 100 % definiert.
Alle Werte außerhalb dieses Intervalls wurden durch NaN ersetzt.

---

💨 Wspd – Windgeschwindigkeit

Die Variable Wspd beschreibt die tägliche durchschnittliche Windgeschwindigkeit in km/h.
Die Verteilung zeigt eine leichte Rechtsschiefe mit einem Skewness-Wert von 1.51.
Der Mittelwert liegt bei 11.34 km/h, der Median bei 10.00 km/h, was auf einen leicht asymmetrischen, aber plausiblen Verlauf hindeutet.

Der Stripplot zeigt, dass die meisten Messwerte zwischen 0 und 45 km/h liegen, mit ausdünnender Streuung nach oben – was meteorologisch zu erwarten ist.
Drei auffällige Extremwerte oberhalb von 150 km/h wurden identifiziert, der Maximalwert liegt bei 176.3 km/h. Solche Werte sind theoretisch möglich (z. B. bei Orkanen), aber im Kontext dieser Analyse nicht repräsentativ.

→ Es wurde ein Cutoff bei 150 km/h gesetzt.
Drei Werte oberhalb dieses Grenzwerts wurden durch NaN ersetzt.

---

🌬️ Pres – Luftdruck

Die Variable Pres beschreibt den täglichen Luftdruck in hPa. Die Verteilung ist mit einer Skewness von –0.04 nahezu symmetrisch, was auf eine ausgewogene, realistische Verteilung hinweist.
Der Mittelwert liegt bei 1015.15 hPa, was nahe am physikalischen Normaldruck (1013 hPa) liegt.

Ein Blick auf die Extremwerte zeigt:

- Minimum: 925.2 hPa – plausibel, z. B. bei Tiefdrucklagen
- Maximum: 1392.1 hPa – stark unrealistisch, deutlich über bekannten Rekorden

→ Es wurde ein Cutoff bei 1100 hPa gesetzt.
Zwei klare Ausreißer oberhalb dieses Grenzwerts wurden durch NaN ersetzt.

---

🌧️ Prcp – Niederschlag

Die Variable Prcp beschreibt die tägliche Niederschlagsmenge in mm. Die Verteilung ist mit einer Skewness von 8.21 stark rechtsschief:

- An über der Hälfte aller Tage wurde kein Niederschlag gemessen (Median: 0.0 mm).
- Einzelne Starkregenereignisse führen zu hohen Maximalwerten (bis zu 462 mm), die jedoch extrem selten sind.

→ Zur Vermeidung einer übermäßigen Verzerrung der Analyse wurde ein Cutoff bei 400 mm gesetzt.
Zwei ungewöhnlich hohe Werte oberhalb dieses Grenzwerts wurden durch NaN ersetzt.

---

💧 Dew – Taupunkt

Die Variable Dew beschreibt den täglichen Taupunkt in °C – also die Temperatur, bei der die Luft mit Wasserdampf gesättigt ist.
Die Werte zeigen eine leicht linksschiefe Verteilung mit einer Skewness von –0.37.
Mittelwert und Median liegen nahe beieinander (9.69 °C bzw. 10.00 °C), was auf eine insgesamt ausgewogene Verteilung hindeutet.

Der Stripplot vermittelt einen „gestückelten“ Eindruck, da etwa zwei Drittel der Werte ganzzahlig sind – vermutlich durch vorverarbeitete oder gerundete Messdaten. Ein gutes Drittel liegt jedoch im nicht-ganzzahligen Bereich, was unterschiedliche Genauigkeitsstufen in den Datenquellen vermuten lässt.

Zur Erkennung potenzieller Ausreißer wurden die physikalisch plausiblen Grenzwerte des Taupunkts berücksichtigt:

- Oberhalb von 32 °C (tropisch-feucht) und
- Unterhalb von –40 °C (polar-trocken) sind reale Messwerte zwar nicht unmöglich, aber extrem selten.

→ Es wurde ein Cutoff bei –40 °C und +32 °C gesetzt. Werte außerhalb dieses Bereichs wurden durch NaN ersetzt.


### Vergleich Anzahl fehlender Werte vor und nach der Ausreißerentfernung (Wetter)

In [ ]:
missing_after_weather = df.isna().sum()
outliers_weather = missing_after_weather - missing_before_weather

outlier_weather_table = pd.DataFrame({
    "Missing Before": missing_before_weather,
    "Missing After": missing_after_weather,
    "Outliers Replaced": outliers_weather
})

# Nur Zeilen anzeigen, in denen tatsächlich Ausreißer ersetzt wurden - alos die Schadstoff-Spalten
outlier_weather_table_filtered = outlier_weather_table[outlier_weather_table["Outliers Replaced"] != 0]

outlier_weather_table_filtered

In [ ]:
missing_before_total_weather = missing_before_weather.sum()
missing_after_total_weather = missing_after_weather.sum()
outliers_total_weather = missing_after_total_weather - missing_before_total_weather
outlier_percent_weather = outliers_total_weather / total_values_before_weather * 100

print(f"Anzahl ersetzter Ausreißer (Wetterdaten): {outliers_total_weather:,}")
print(f"Prozentualer Anteil der ersetzten Ausreißer: {outlier_percent_weather:.4f}%")


Durch das Entfernen klar identifizierter Ausreißer **in den Wettervariablen** wurden lediglich 0.001 % der Datenpunkte ersetzt.
Der Datensatz wurde dadurch kaum reduziert, ist aber nun robuster gegenüber Verzerrungen, was insbesondere für Machine-Learning-Modelle, Visualisierungen und Clusteranalysen von Vorteil ist.

In [ ]:
df.to_csv("data/cleaned_air_quality_data_2025_03_25.csv", index=False)

# 3. Korrelationsanalysen

## 3.1 Korrelationsmatrix (Pearson)

Die Matrix enthält alle numerischen Features aus dem Datensatz. Als Maßstab für die Stärke einer Korrelation werden folgende Werte angesetzt:

\> 0.8 = strong correlation

0.4 - 0.8 = moderate correlation

< 0.4 = weak correlation

In [ ]:

# calculate correlation matrix (Pearson)
corr_matrix = df.select_dtypes(include=['number']).corr()

# Mask upper triangle
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

# Display heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt=".2f", cmap="coolwarm", center=0, linewidths=0.5)
plt.title("Feature Correlations (Pearson)");


In [ ]:
# Show only strong and moderate correlations (>|0.4|); leave out main diagonal (1.0)

# Calculate matrix
corr_matrix = df.select_dtypes(include=['number']).corr()

# extract only strong and moderate correlations (>|0.4|); leave out main diagonal (1.0)
strong_corrs = corr_matrix[(corr_matrix.abs() > 0.4) & (corr_matrix.abs() < 1.0)]

# Convert df to long list (.stack) and reset index
strong_corrs = strong_corrs.stack().reset_index()
strong_corrs.columns = ["Feature 1", "Feature 2", "Korrelation"]

# remove redundant rows (note: the "<"-sign here refers to alphabetic order of feature names, not to numbers of any kind!)
strong_corrs = strong_corrs.loc[strong_corrs["Feature 1"] < strong_corrs["Feature 2"]]

strong_corrs

## Interpretation der Korrelationen und Vorschläge für weitere Analysen

Die Korrelationsmatrix zeigt mehrere starke und inhaltlich gut erklärbare Zusammenhänge zwischen den numerischen Variablen. Besonders deutlich sind folgende Muster:

**Feinstaub- und Stickstoffdioxidwerte hängen zusammen:**
Es besteht eine starke Korrelation zwischen PM10 und PM2.5 (r = 0.83), was plausibel ist, da PM2.5 eine Teilmenge von PM10 ist. Zusätzlich korreliert NO₂ moderat mit beiden Feinstaubkomponenten (r ≈ 0.42–0.48), was auf gemeinsame Emissionsquellen wie Verkehr oder Industrie hindeutet.

**Temperaturvariablen sind stark untereinander korreliert:**
Die Tagesmitteltemperatur (Tavg) steht in sehr engem Zusammenhang mit Tmin und Tmax (r ≈ 0.97). Auch Tmin und Tmax selbst sind hoch korreliert (r = 0.90). Das ist mathematisch und physikalisch naheliegend und spricht dafür, nicht alle drei Variablen gleichzeitig zu verwenden, um Redundanz zu vermeiden.

**Der Taupunkt (Dew) korreliert stark mit Temperatur:**
Die stärkste Korrelation liegt zwischen Dew und Tmin (r = 0.87), gefolgt von Tavg (r = 0.82). Dies spiegelt wider, dass die Luftfeuchtigkeit – und damit der Taupunkt – eng mit der Umgebungstemperatur zusammenhängt.

**Einige schwächer negative Korrelationen deuten auf atmosphärische Zusammenhänge hin:**
Der Luftdruck (Pres) korreliert moderat negativ mit Dew (r = –0.43) und Tmin (r = –0.42), was mit typischen meteorologischen Prozessen in Zusammenhang stehen kann (z. B. feuchtwarme Luft in Tiefdruckgebieten).

---

Diese Ergebnisse helfen dabei, hoch korrelierte bzw. redundante Variablen zu erkennen und gezielt für weitere Analysen (z. B. Clusteranalyse oder Modellierung) geeignete Features auszuwählen.

Für viele Verfahren, wie z. B. Clustering oder Regressionsmodelle, ist es ratsam, von stark korrelierten Variablen jeweils nur eine zu verwenden, um Verzerrungen oder sogenannte Multikollinearität zu vermeiden.

Alternativ können Hauptkomponentenanalyse (PCA) oder andere dimensionalitätsreduzierende Verfahren genutzt werden, um mehrere stark korrelierte Variablen zu einer gemeinsamen Komponente zusammenzufassen, ohne wesentliche Information zu verlieren.


# 3.2 Pairplot der wichtigsten Feature-Paare

Für eine Pairplot-Analyse werden nur bestimmte Features ausgewählt, bei denen entweder aufgrund von Weltwissen oder der berechneten Korrelationswerte relevante Zusammenhänge zu erwarten sind. Diese sind: PM25, PM10, NO₂, O3, CO, Tavg, Humidity, Dew, Pres.

Von diesen werden wiederum nur einzelne Paare getestet: PM10-PM25, NO₂-PM10, NO₂-PM25, Humidity-PM25, O3-Tavg, Tmin-Dew, Pres-Dew, Pres-Tmin.

Die Pairplots werden auf einem repräsentativen Sample von 2000 Datenpunkten berechnet.

In [ ]:
features = ["Pm25", "Pm10", "No2", "O3", "Co", "Tavg", "Humidity", "Dew", "Pres"]
sample_df = df[features].dropna().sample(n=2000, random_state=42)
sns.pairplot(sample_df, diag_kind="kde", plot_kws={"alpha": 0.4, "s": 15})

Die Histogramme im Pairplot bestätigen die zuvor berechnete Schiefe vieler Variablen:

Schadstoffe wie Pm25, NO2 und O3 sind erwartungsgemäß rechtsschief verteilt, während Temperatur (Tavg), Luftfeuchtigkeit (Humidity) und Taupunkt (Dew) eine leichte Linksschiefe zeigen. Der Luftdruck (Pres) hebt sich durch seine nahezu normalverteilte Form ab.

# Überprüfung einzelner Variablenpaare mit linearer Regression und/oder LOWESS

Um die Art des Zusammenhangs (lienar oder nicht) zwischen zwei Features eindeutig zu identifizieren, werden sowohl lineare Regression als auch LOWESS (Local Weighted Scatterplot Smoothing) angewendet.

Die Samplegröße beträgt jeweils 3000 Datenpunkte. Zur Replizierbarkeit wird ein Random State von 42 festgelegt. NaN-Werte werden für alle Berechnungen entfernt (weil Seaborn damit nicht umgehen kann).

## Zusammenhang zwischen Pm10 und Pm25

Pearson-Koeffizient: 0,83

Pm25 ist eine Teilmenge von Pm10. Sind beide Features relevant für spätere Modellierungen oder ist eines davon redundant?

In [ ]:
subset = df[["Pm10", "Pm25"]].dropna().sample(n=3000, random_state=42)

plt.figure(figsize=(8, 5))
sns.regplot(data=subset, x="Pm10", y="Pm25", lowess=True,
            scatter_kws={"alpha": 0.3, "s": 15},
            line_kws={"color": "crimson"})

plt.title("LOWESS-Regressionskurve: PM10 vs. PM2.5")
plt.xlabel("PM10-Konzentration (µg/m³)")
plt.ylabel("PM2.5-Konzentration (µg/m³)")
plt.grid(True);

Die LOWESS-Analyse bestätigt den erwarteten sehr starken linearen Zusammenhang zwischen PM10 und PM2.5.

Im zentralen Bereich steigt PM2.5 nahezu proportional zu PM10, mit nur einer leichten Abflachung bei höheren PM10-Werten.
Diese minimale Abweichung könnte darauf hindeuten, dass bei sehr hoher Feinstaubbelastung der relative Anteil von groben Partikeln (PM10 ohne PM2.5) etwas zunimmt.

Aufgrund der wenigen Datenpunkte in den extremen Bereichen kann für diesen keine Belastbare Aussage gemacht werden.

## Zusammenhang zwischen NO2 und PM10

Pearson-Koeffizient: 0,42

Der Zusammenhang sollte ähnlich sein wie der zwischen No2 und PM25, aber um abzuschätzen, ob für spätere Analysen ein Feature redundant ist, gehen wir mit LOWESS sicher:

In [ ]:
subset = df[["No2", "Pm10"]].dropna().sample(n=3000, random_state=42)

plt.figure(figsize=(8, 5))
sns.regplot(data=subset, x="No2", y="Pm10", lowess=True,
            scatter_kws={"alpha": 0.3, "s": 15},
            line_kws={"color": "darkorange"})

plt.title("LOWESS-Regressionskurve für NO₂ und PM10")
plt.xlabel("NO₂-Konzentration (µg/m³)")
plt.ylabel("PM10-Konzentration (µg/m³)")
plt.grid(True);

Die LOWESS-Analyse des Zusammenhangs zwischen NO₂ und PM10 zeigt einen insgesamt positiven Trend, der im Bereich bis etwa 40 µg/m³ NO₂ weitgehend linear verläuft. Ab etwa 40 µg/m³ NO₂ wird der Zusammenhang flacher, was auf eine Sättigung oder zunehmende Streuung hinweisen könnte. In diesem Bereich sind jedoch nur wenige Datenpunkte vorhanden, sodass der Verlauf der Kurve dort nicht belastbar interpretiert werden sollte.

Eine lineare Regression könnte im unteren Bereich sinnvoll sein, sollte aber auf den zentralen Wertebereich beschränkt werden.

## Zusammenhang zwischen NO2 und PM25

Pearson-Koeffizient: 0,48

In [ ]:
sample_df = df[["No2", "Pm25"]].dropna().sample(n=3000, random_state=42)

plt.figure(figsize=(8, 5))
sns.regplot(data=sample_df, x="No2", y="Pm25",
            scatter_kws={"alpha":0.3, "s":15},
            line_kws={"color": "darkred"})

plt.title("Zusammenhang zwischen NO₂ und PM2.5 (Stichprobe)")
plt.xlabel("NO₂-Konzentration (µg/m³)")
plt.ylabel("PM2.5-Konzentration (µg/m³)")
plt.grid(True)

Der regplot zwischen NO₂ und PM2.5 zeigt eine positive lineare Tendenz, was plausibel erscheint, da beide Schadstoffe häufig durch Verkehr oder industrielle Prozesse freigesetzt werden.
Aufgrund der breiten Streuung und fehlender Kontextvariablen ist die Regressionslinie jedoch nicht als kausales Modell, sondern lediglich als explorative Visualisierung einer Korrelation zu verstehen.
Der Konfidenzbereich ist nur für den Bereich gut, in dem die meisten Messwerte liegen. Problem: Heteroskadastizität.

In [ ]:
subset = df[(df["No2"] < 40) & (df["Pm25"] < 200)][["No2", "Pm25"]].dropna().sample(n=3000, random_state=42)

plt.figure(figsize=(8, 5))
sns.regplot(data=subset, x="No2", y="Pm25",
            scatter_kws={"alpha": 0.3, "s": 15},
            line_kws={"color": "darkblue"})

plt.title("Regression innerhalb des zentralen Wertebereichs (No2 < 40, Pm25 < 200)")
plt.xlabel("NO₂-Konzentration (µg/m³)")
plt.ylabel("PM2.5-Konzentration (µg/m³)")
plt.grid(True);

Nimmt man nur den zentralen Wertebereich heraus, ist eine lineare Verteilung sichtbar, und auch das Konfidenzintervall ist "akzeptabel".

In [ ]:
subset = df[["No2", "Pm25"]].dropna().sample(n=3000, random_state=42)

plt.figure(figsize=(8, 5))
sns.regplot(data=subset, x="No2", y="Pm25", lowess=True,
            scatter_kws={"alpha": 0.3, "s": 15},
            line_kws={"color": "mediumvioletred"})

plt.title("LOWESS-Regressionskurve für NO₂ und PM2.5")
plt.xlabel("NO₂-Konzentration (µg/m³)")
plt.ylabel("PM2.5-Konzentration (µg/m³)")
plt.grid(True)

Die Anwendung von LOWESS auf den gesamten Wertebereich von NO₂ und PM2.5 zeigt, dass der Zusammenhang nicht durchgängig linear ist.
Besonders im oberen Wertebereich (ab ~70 µg/m³ NO₂) ist die Punktverteilung zu dünn und uneinheitlich, was zu starken Schwankungen in der Regressionskurve führt.
Im zentralen Wertebereich (NO₂ < 40, PM2.5 < 200) zeigt sich hingegen ein stabiler, positiver Trend.
Für lineare Analysen ist daher eine Einschränkung auf diesen Bereich sinnvoll.

--> Der Zusammenhang ist also ähnlich wie der zwischen NO₂ und PM10.

## Zusammenhang zwischen Luftfeuchtigkeit (Humidity) und mittelgroßen Feinstaubpartikeln (Pm25)

Pearson-Koeffizient: 0,13



In [ ]:
subset = df[["Humidity", "Pm25"]].dropna().sample(n=3000, random_state=42)

plt.figure(figsize=(8, 5))
sns.regplot(data=subset, x="Humidity", y="Pm25", lowess=True,
            scatter_kws={"alpha": 0.3, "s": 15},
            line_kws={"color": "teal"})

plt.title("LOWESS-Regressionskurve: Luftfeuchtigkeit vs. PM2.5")
plt.xlabel("Relative Luftfeuchtigkeit (%)")
plt.ylabel("PM2.5-Konzentration (µg/m³)")
plt.grid(True);

Die LOWESS-Regression zwischen Luftfeuchtigkeit (Humidity) und PM2.5 zeigt, wie erwartet, einen klar negativen Zusammenhang.
Bei niedriger Luftfeuchtigkeit (unter 20 %) liegen die PM2.5-Werte durchschnittlich bei etwa 70 µg/m³, während sie bei sehr hoher Luftfeuchtigkeit (über 90 %) auf unter 50 µg/m³ sinken.

Dieser Verlauf ist nicht ganz linear, sondern flacht bei zunehmender Feuchtigkeit ab – ein Hinweis auf sättigende Effekte oder natürliche Begrenzungen.

Eine lineare Regression wäre hier möglich, aber nicht unbedingt angemessen – LOWESS zeigt die tatsächliche Struktur des Zusammenhangs besser.

## Zusammenhang zwischen Ozonwert (O3) und Durchnittstemperatur (Tavg)

Pearson-Koeffizient: 0,19

In [ ]:
subset = df[["O3", "Tavg"]].dropna().sample(n=3000, random_state=42)
plt.figure(figsize=(8, 5))
sns.regplot(data=subset, x="Tavg", y="O3", lowess=True,
            scatter_kws={"alpha": 0.3, "s": 15},
            line_kws={"color": "darkgreen"})

plt.title("Nichtlinearer Zusammenhang zwischen Temperatur (Tavg) und Ozon (O3)")
plt.xlabel("Durchschnittstemperatur (°C)")
plt.ylabel("Ozonkonzentration (µg/m³)")
plt.grid(True);

Die Analyse des Zusammenhangs zwischen Temperatur (Tavg) und Ozonkonzentration (O3) mit Hilfe einer LOWESS-Regression zeigt einen klar nichtlinearen Verlauf:

- Bei Temperaturen zwischen 0 und etwa 20 °C steigt die Ozonkonzentration tendenziell an – ein plausibler Effekt durch sonnengetriebene Bildung.
- Ab ca. 20 °C sinkt die Konzentration jedoch wieder.

Dieser geknickte Verlauf weist darauf hin, dass eine lineare Regressionsanalyse hier nicht geeignet ist, um den Zusammenhang korrekt zu modellieren.

--> Stichwort: „Ozon-Peak-Temperatur-Phänomen“

## Zusammenhang zwischen Tagestiefsttemperatur (Tmin) und Taupunkt (Dew)

Pearson-Koeffizient: 0.87

Die Temperatur beeinflusst den Taupunkt. Der Taupunkt ist der Punkt, bei dem die Luft gesättigt ist und Wasserdampf kondensiert. Er ist immer kleiner oder gleich der aktuellen Lufttemperatur. Er nähert sich der Lufttemperatur vor allem nachts, wenn es abkühlt. Daher ist der Zusammenhang mit Tmin am stärksten.

In [ ]:
subset = df[["Tmin", "Dew"]].dropna().sample(n=3000, random_state=42)

plt.figure(figsize=(8, 5))
sns.regplot(data=subset, x="Tmin", y="Dew", lowess=True,
            scatter_kws={"alpha": 0.3, "s": 15},
            line_kws={"color": "mediumseagreen"})

plt.title("LOWESS-Regressionskurve: Tmin vs. Taupunkt (Dew)")
plt.xlabel("Tiefsttemperatur (°C)")
plt.ylabel("Taupunkt (°C)")
plt.grid(True);

Es zeigt sich ein deutlicher linearer Zusammenhang ohne nennenswerte Knicke.

Vergleich von LOWESS und linearer Regressionlinie:

In [ ]:
subset = df[["Tmin", "Dew"]].dropna().sample(n=3000, random_state=42)

plt.figure(figsize=(8, 5))

# LOWESS
sns.regplot(data=subset, x="Tmin", y="Dew", lowess=True,
            scatter_kws={"alpha": 0.3, "s": 15},
            line_kws={"color": "mediumseagreen", "label": "LOWESS"})

# Lineare Regression
sns.regplot(data=subset, x="Tmin", y="Dew", lowess=False,
            scatter=False,
            line_kws={"color": "orangered", "linestyle": "--", "label": "Linear"})

plt.title("Tmin vs. Taupunkt (LOWESS & Lineare Regression)")
plt.xlabel("Tiefsttemperatur (°C)")
plt.ylabel("Taupunkt (°C)")
plt.legend()
plt.grid(True);


Tiefsttemperatur (Tmin) und Taupunkt (Dew) sind hoch korreliert (r = 0.87).

Ein LOWESS-Plot offenbart einen weitgehend linearen Anstieg des Taupunkts mit zunehmender Temperatur, insbesondere im Bereich unter 0 °C. In höheren Temperaturbereichen wird die Steigung etwas flacher, was auf eine schwächer werdende Kopplung hinweisen könnte.

Zum Vergleich wurde zusätzlich eine lineare Regressionslinie mit Konfidenzintervall geplottet. Sie verläuft insgesamt flacher als die LOWESS-Kurve. Das Konfidenzband ist sehr schmal, was auf eine geringe Streuung und hohe Vorhersagbarkeit hinweist.

Der Vergleich zeigt, dass eine lineare Regression im zentralen Bereich zwar möglich, aber nicht ideal ist.
LOWESS eignet sich, um die **leichte** Nichtlinearität sichtbar zu machen und kann für explorative Zwecke empfohlen werden.

## Zusammenhang zwischen Luftdruck (Pres) und Taupunkt (Dew)

Korrelationskoeffizient: -0,43

In [ ]:
subset = df[["Pres", "Dew"]].dropna().sample(n=3000, random_state=42)

plt.figure(figsize=(8, 5))
sns.regplot(data=subset, x="Pres", y="Dew", lowess=True,
            scatter_kws={"alpha": 0.3, "s": 15},
            line_kws={"color": "darkorange"})

plt.title("LOWESS-Regressionskurve: Luftdruck vs. Taupunkt")
plt.xlabel("Luftdruck (hPa)")
plt.ylabel("Taupunkt (°C)")
plt.grid(True)

Die LOWESS-Analyse zwischen Luftdruck (Pres) und Taupunkt (Dew) zeigt einen nichtlinearen, umgekehrt U-förmigen Zusammenhang.
Bei niedrigen Druckwerten steigt der Taupunkt zunächst an, flacht im mittleren Bereich ab und fällt schließlich bei hohem Druck steil ab.

Dies unterstützt die meteorologische Annahme, dass Tiefdruck mit feuchterer Luft (höherem Taupunkt) und Hochdruck mit trockenerer Luft (niedrigerem Taupunkt) einhergeht.

## Zusammenhang zwischen Tagestiefsttemperatur (Tmin) und Luftdruck (Pres)

Korrelationskoeffizient: -0,42

In [ ]:
subset = df[["Pres", "Tmin"]].dropna().sample(n=3000, random_state=42)

plt.figure(figsize=(8, 5))
sns.regplot(data=subset, x="Pres", y="Tmin", lowess=True,
            scatter_kws={"alpha": 0.3, "s": 15},
            line_kws={"color": "darkorange"})

plt.title("LOWESS-Regressionskurve: Luftdruck vs. Tiefsttemperatur")
plt.xlabel("Luftdruck (hPa)")
plt.ylabel("Tiefsttemperatur (°C)")
plt.grid(True)

Die LOWESS-Analyse zwischen Luftdruck (Pres) und Tiefsttemperatur (Tmin) zeigt einen symmetrisch gebogenen Zusammenhang, ähnlich wie bei Pres und Dew.

- In Bereichen mit niedrigem Druck steigen die nächtlichen Tiefsttemperaturen zunächst leicht an, was auf eine isolierende Wirkung von Wolken und Wetteraktivität bei Tiefdruck hindeutet.
- Ab einem Druck von etwa 1008 hPa kehrt sich der Effekt um: Bei hohem Druck sinken die Tiefsttemperaturen, vermutlich durch klare Nächte mit stärkerer Abkühlung.

Die Kurve ist harmonisch geformt und belegt eine meteorologisch plausible, nichtlineare Beziehung.

# Abschließende Bewertung der Korrelationen und der Datenstruktur

Die ausführliche Analyse der Korrelationen – unterstützt durch klassische Korrelationsmatrizen, Regressionsplots und LOWESS-Kurven – hat gezeigt, dass die betrachteten Variablen sinnvolle, teils lineare, teils nichtlineare Zusammenhänge aufweisen, die physikalisch, meteorologisch oder umweltbezogen plausibel erklärbar sind.

Dabei konnten sowohl bekannte Beziehungen (z. B. zwischen Feinstaub und Stickstoffdioxid oder zwischen Taupunkt und Temperatur) als auch weniger offensichtliche, aber nachvollziehbare Muster (z. B. der Einfluss des Luftdrucks auf Temperatur und Feuchtigkeit) sichtbar gemacht werden.

Insgesamt deutet die Korrelationenanalyse darauf hin, dass es sich um natürliche, glaubwürdige und konsistente Umweltdaten handelt, die keine Anzeichen künstlicher Verzerrung oder unplausibler Artefakte zeigen.

Damit ist der Datensatz in seiner bereinigten Form eine solide Grundlage für weiterführende Machine-Learning-Modelle. Die erkannten Strukturen können gezielt genutzt werden, um sinnvolle Features zu definieren, Zusammenhänge zu explorieren und vorhersagende Modelle zu entwickeln, die auf realweltlichen Mustern basieren.